<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [6]</a>'.</span>

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import os
from time import time
import argparse

# Defining SQL table using pandas.

In [ ]:
user = ""
password = ""
host = ""
port = 0
db = ""
table_name = ""
csv_name = ""

In [ ]:
# Parameters
user = "root"
password = "root"
host = "localhost"
port = 5432
db = "ny_taxi"
table_name = "yellow_taxi_data"
csv_name = "yellow_tripdata_2021-01.csv"


In [ ]:
params = {"user":user,"password":password,"host":host,"port":port,"db":db,"table_name":table_name,"csv_name":csv_name}

In [ ]:
def main(params):
    user = params['user']
    password = params['password']
    host = params['host']
    port = params['port']
    db = params['db']
    table_name = params['table_name']
    csv_name = params['csv_name']
    
    print(user)

    engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')

    df_iter = pd.read_csv(csv_name, iterator=True, chunksize=100000)

    df = next(df_iter)

    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

    df.head(n=0).to_sql(name=table_name, con=engine, if_exists='replace')

    df.to_sql(name=table_name, con=engine, if_exists='append')

    while True:
        t_start = time()

        df = next(df_iter)

        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

        df.to_sql(name=table_name, con=engine, if_exists='append')

        t_end = time()

        print('inserted another chunk, took %.3f second' % (t_end - t_start))

In [ ]:
main (params)